# ⭐ Complete Model Validation Workflow

<div style="background-color: #fff3cd; padding: 20px; border-radius: 10px; border-left: 6px solid #ffc107;">
<h3 style="color: #856404; margin-top: 0;">🌟 MAIN DEMO - This is the most important notebook!</h3>
<p style="color: #856404;">This notebook demonstrates <b>all the power of DeepBridge</b> in a complete end-to-end workflow.<br>
Get ready to see magic happen! ✨</p>
</div>

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3; margin-top: 20px;">
<b>📓 Notebook Information</b><br>
<b>Level:</b> Beginner-Intermediate<br>
<b>Estimated Time:</b> 20-30 minutes<br>
<b>Prerequisites:</b> 01_first_steps.ipynb (recommended)<br>
<b>Dataset:</b> Credit Scoring (synthetic)
</div>

## 📖 The Story

You are a **Data Scientist** at a bank and need to validate a **Credit Scoring** model before putting it into production.

The model decides who gets credit - a decision that impacts lives!

### Requirements
- ✅ Model must be **robust** (resist manipulation attempts)
- ✅ Model must be **fair** (no bias by gender, race, etc.)
- ✅ Must be in **compliance** with regulations (EEOC, Fair Lending Laws)
- ✅ Needs **professional reports** for auditing

**Challenge:** Do all this in less than 30 minutes! ⏱️

**Solution:** DeepBridge! 🚀

## 🎯 What you will do

1. 📊 Load and prepare credit scoring data
2. 🤖 Train a RandomForest model
3. 📦 Create DBDataset (data container)
4. 🔬 Create Experiment (orchestrator)
5. 🧪 Run validation tests
   - Robustness (resistance to perturbations)
   - Fairness (bias analysis)
6. 📊 Generate professional HTML reports
7. ✅ Decide: Deploy or not?

**All this with DeepBridge in ~30 lines of code!**

---
# PART 1: Data Preparation 📊

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# DeepBridge - The star of the show!
from deepbridge import DBDataset, Experiment

# Settings
np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("✅ Imports completed!")

### Create Synthetic Credit Scoring Dataset

In [ ]:
# Simulate realistic credit scoring dataset
n_samples = 2000

# Demographic features
age = np.random.normal(40, 12, n_samples).clip(18, 80)
gender = np.random.choice(['M', 'F'], n_samples)
education = np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], n_samples, p=[0.3, 0.4, 0.2, 0.1])

# Financial features
income = np.random.gamma(5, 10000, n_samples)
credit_score = np.random.normal(650, 80, n_samples).clip(300, 850)
debt_ratio = np.random.beta(2, 5, n_samples) * 100
employment_length = np.random.exponential(5, n_samples).clip(0, 40)

# Target (credit approval)
# Logic: high income + high credit score + low debt = approved
score = (income / 10000) * 0.3 + (credit_score / 100) * 0.4 + (100 - debt_ratio) * 0.01 + employment_length * 0.05
default_prob = 1 / (1 + np.exp(-0.1 * (score - 50)))  # Sigmoid
approved = (default_prob > 0.5).astype(int)

# Create DataFrame
df = pd.DataFrame({
    'age': age,
    'gender': gender,
    'education': education,
    'income': income,
    'credit_score': credit_score,
    'debt_ratio': debt_ratio,
    'employment_length': employment_length,
    'approved': approved
})

print(f"📊 Dataset created: {df.shape}")
print(f"\n🎯 Approval rate: {df['approved'].mean():.1%}")
df.head()

### Quick EDA

In [ ]:
# Visualize approval distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# By gender
df.groupby('gender')['approved'].mean().plot(kind='bar', ax=axes[0], color=['skyblue', 'lightcoral'])
axes[0].set_title('Approval Rate by Gender', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Approval Rate')
axes[0].set_ylim(0, 1)
axes[0].axhline(y=0.5, color='red', linestyle='--', label='Average')

# By education
df.groupby('education')['approved'].mean().plot(kind='bar', ax=axes[1], color='lightgreen')
axes[1].set_title('Approval Rate by Education', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Approval Rate')
axes[1].set_ylim(0, 1)
axes[1].axhline(y=0.5, color='red', linestyle='--', label='Average')

plt.tight_layout()
plt.show()

print("💡 Observation: Let's check if there's bias in these variables!")

---
# PART 2: Model Training 🤖

In [ ]:
# Prepare data
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()
le_gender = LabelEncoder()
le_education = LabelEncoder()
df_encoded['gender'] = le_gender.fit_transform(df_encoded['gender'])
df_encoded['education'] = le_education.fit_transform(df_encoded['education'])

# Separate X and y
X = df_encoded.drop('approved', axis=1)
y = df_encoded['approved']

# Manual split for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"📊 Data split:")
print(f"   Train: {X_train.shape}, Test: {X_test.shape}")

### Train RandomForest

In [ ]:
# Train model
clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=20,
    random_state=42,
    n_jobs=-1
)

print("🤖 Training RandomForest...")
clf.fit(X_train, y_train)

# Evaluate
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

print(f"\n✅ Model trained!")
print(f"   Accuracy (train): {train_score:.3f}")
print(f"   Accuracy (test): {test_score:.3f}")
print(f"   Overfitting: {(train_score - test_score):.3f}")

In [ ]:
# Classification report
y_pred = clf.predict(X_test)
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Rejected', 'Approved']))

<div style="background-color: #d1ecf1; padding: 15px; border-radius: 5px; border-left: 5px solid #0c5460;">
<b>📌 Checkpoint:</b> Model trained with ~80% accuracy. Looks good, but is it enough?
<br>🤔 Is it robust? Is it fair? Let's find out with DeepBridge!
</div>

---
# PART 3: Create DBDataset 📦

Now the magic begins! Let's use DeepBridge.

In [ ]:
# Create DBDataset with model
dataset = DBDataset(
    data=df_encoded,
    target_column='approved',
    model=clf,  # ← Integrated model!
    test_size=0.2,
    random_state=42,
    dataset_name='Credit Scoring Model'
)

print("✅ DBDataset created!")
print(f"\n📊 Information:")
print(f"   Train: {len(dataset.train_data)} samples")
print(f"   Test: {len(dataset.test_data)} samples")
print(f"   Features: {dataset.features}")
print(f"   Categorical: {dataset.categorical_features}")
print(f"   Predictions available: {dataset.test_predictions is not None}")

<div style="background-color: #d4edda; padding: 10px; border-radius: 5px; border-left: 5px solid #28a745;">
<b>✅ Magic!</b> DBDataset automatically detected categorical features and generated predictions!
</div>

---
# PART 4: Create Experiment 🔬

The **Experiment** is the orchestrator that coordinates all tests.

In [ ]:
# Create Experiment
exp = Experiment(
    dataset=dataset,
    experiment_type='binary_classification',  # Problem type
    random_state=42
)

print("✅ Experiment created!")
print(f"\n📊 Initial Metrics:")
for metric, value in list(exp.initial_results.items())[:5]:
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.3f}")
    else:
        print(f"   {metric}: {value}")

---
# PART 5: Run Validation Tests 🧪

Here begins the TRUE validation!

## 5.1 Robustness Test (Quick)

In [ ]:
print("🔍 Running ROBUSTNESS test (config='quick')...")
print("⏱️ This should take ~1-2 minutes\n")

# Run test
rob_results = exp.run_test('robustness', config_name='quick')

print("\n✅ Robustness Test Completed!")
print(f"\n📊 Results:")
print(f"   Robustness Score: {rob_results.get('robustness_score', 'N/A')}")
print(f"   Degradation: {rob_results.get('performance_degradation', 'N/A')}")
print(f"\n💡 Interpretation:")
print(f"   Score close to 1.0 = Very robust ✅")
print(f"   Score < 0.7 = Warning, sensitive model! ⚠️")

## 5.2 Fairness Analysis (CRITICAL!) ⚖️

<div style="background-color: #fff3cd; padding: 15px; border-radius: 5px; border-left: 5px solid #ffc107;">
<b>⚠️ CRITICAL:</b> For Credit Scoring, fairness is MANDATORY by law!
<br>We need to verify compliance with Fair Lending Laws and EEOC.
</div>

In [ ]:
# Automatically detect sensitive attributes
print("🔍 Detecting sensitive attributes...\n")
sensitive_attrs = Experiment.detect_sensitive_attributes(dataset, threshold=0.7)
print(f"📋 Detected sensitive attributes: {sensitive_attrs}")

# Note: 'gender' should be detected automatically!

In [ ]:
# Create new experiment with protected attributes
exp_fair = Experiment(
    dataset=dataset,
    experiment_type='binary_classification',
    protected_attributes=['gender'],  # Protected attribute
    random_state=42
)

print("✅ Experiment with fairness configured!")

In [ ]:
print("⚖️ Running FAIRNESS ANALYSIS (config='quick')...")
print("⏱️ This should take ~2-3 minutes\n")

# Run fairness tests
fairness_result = exp_fair.run_fairness_tests(config='quick')

print("\n✅ Fairness Analysis Completed!")
print(f"\n⚖️ EEOC Verification (80% rule):")
if fairness_result.passes_eeoc_compliance():
    print("   ✅ PASS - Model is in compliance!")
else:
    print("   ❌ FAIL - Model is NOT in compliance!")
    print("   🚨 ACTION REQUIRED: Review model before deploy!")

## 5.3 Run All Tests (Optional)

In [ ]:
# Uncomment to run ALL tests
# print("🧪 Running ALL tests (config='quick')...")
# print("⏱️ This may take 5-10 minutes\n")
# 
# all_results = exp.run_tests(config_name='quick')
# 
# print("\n✅ All tests completed!")
# print(f"   Tests executed: {list(all_results._test_results.keys())}")

print("(Commented out to save time - run it if you want to see everything!)") 

---
# PART 6: Generate Reports 📊

DeepBridge generates professional HTML reports automatically!

In [ ]:
import os
output_dir = 'reports'
os.makedirs(output_dir, exist_ok=True)

print("📊 Generating HTML Reports...\n")

# Robustness report
rob_report = os.path.join(output_dir, 'credit_scoring_robustness.html')
exp.save_html('robustness', rob_report, model_name='Credit Scoring Model')
print(f"✅ Robustness Report: {rob_report}")

# Fairness report
fair_report = os.path.join(output_dir, 'credit_scoring_fairness.html')
fairness_result.save_html(fair_report, model_name='Credit Scoring Model')
print(f"✅ Fairness Report: {fair_report}")

print(f"\n🎉 HTML reports generated!")
print(f"📂 Open the files in your browser to visualize")

<div style="background-color: #d4edda; padding: 15px; border-radius: 5px; border-left: 5px solid #28a745;">
<b>🎉 Incredible!</b> Professional HTML reports ready for presentation and auditing!
<br><br>
<b>Report features:</b>
<ul>
<li>✅ Interactive charts (Plotly)</li>
<li>✅ Detailed tables</li>
<li>✅ Compliance metrics</li>
<li>✅ Ready for stakeholders</li>
</ul>
</div>

---
# PART 7: Deployment Decision ✅

Let's decide if the model is ready for production.

In [ ]:
print("="*60)
print("📋 DEPLOYMENT APPROVAL CHECKLIST")
print("="*60)

# Criteria
test_accuracy = test_score
robustness_ok = rob_results.get('robustness_score', 0) > 0.75
fairness_ok = fairness_result.passes_eeoc_compliance()
accuracy_ok = test_accuracy > 0.75

print(f"\n1. Accuracy > 75%: {'✅' if accuracy_ok else '❌'} ({test_accuracy:.1%})")
print(f"2. Robustness Score > 0.75: {'✅' if robustness_ok else '❌'} ({rob_results.get('robustness_score', 0):.3f})")
print(f"3. EEOC Compliance: {'✅' if fairness_ok else '❌'}")

# Final decision
all_pass = accuracy_ok and robustness_ok and fairness_ok

print("\n" + "="*60)
if all_pass:
    print("🎉 DECISION: MODEL APPROVED FOR PRODUCTION!")
    print("✅ All criteria met")
    print("✅ Compliance guaranteed")
    print("✅ Complete documentation")
else:
    print("⚠️ DECISION: MODEL NEEDS REVISION")
    print("❌ Some criteria not met")
    print("🔄 Recommendation: Adjust model and re-validate")
print("="*60)

---
# 🎉 CONCLUSION

## What you just did

In less than **30 minutes**, you:

1. ✅ Created a credit scoring dataset
2. ✅ Trained a RandomForest model
3. ✅ Created a DBDataset in 1 line
4. ✅ Created an Experiment in 1 line
5. ✅ Ran validation tests:
   - Robustness ✅
   - Fairness ✅
6. ✅ Generated professional HTML reports
7. ✅ Made decision based on objective criteria

## Without DeepBridge vs With DeepBridge

| Task | Without DeepBridge | With DeepBridge |
|------|-------------------|----------------|
| Robustness Tests | 2-3 days | 2 minutes |
| Fairness Analysis | 1-2 weeks | 3 minutes |
| Reports | 1-2 days | Automatic |
| Compliance | Manual, error-prone | Automatic verification |
| **TOTAL** | **~3-4 weeks** | **~30 minutes** |

## DeepBridge ROI

```
⏱️ Time saved: ~3 weeks per model
💰 Cost saved: ~$10,000+ in work hours
✅ Compliance guaranteed: Reduces legal risk
📊 Professional reports: Impresses stakeholders
```

## 🎯 Next Steps

### Deep dive into specific topics:

📘 **DBDataset:**
- `02_dbdataset/01_simple_loading.ipynb`
- `02_dbdataset/03_model_integration.ipynb`

📘 **Specific Tests:**
- `03_validation_tests/02_complete_robustness.ipynb` - Robustness in depth
- `04_fairness/02_complete_fairness_analysis.ipynb` ⭐⭐ - 15 fairness metrics

📘 **Use Cases:**
- `05_use_cases/01_credit_scoring.ipynb` ⭐⭐⭐ - Complete real case
- `05_use_cases/02_medical_diagnosis.ipynb` - Critical application

### Challenge:

💪 **Apply to your own model!**
1. Replace data with your dataset
2. Train your model
3. Run the same tests
4. Generate reports
5. Make data-driven decisions!

---
<div style="background-color: #e3f2fd; padding: 20px; border-radius: 10px; text-align: center;">
<h2 style="color: #1976d2; margin-top: 0;">🎊 Congratulations! 🎊</h2>
<p style="font-size: 18px; color: #1565c0;">
You saw the power of <b>DeepBridge</b> in action!<br>
Professional model validation in minutes, not weeks.
</p>
<p style="font-size: 16px; color: #1976d2;">
Continue exploring the other notebooks to master all the functionalities!
</p>
</div>